In [2]:
import pandas as pd
import numpy as np
from scipy import stats as stt
import streamlit as st
import matplotlib.pyplot as plt
import plotly.express as px
import altair as alt

In [3]:
cars = pd.read_csv(r"C:\Personal Files\Coding\GitHub\sprint-4-project\vehicles_us.csv", sep=',', decimal='.')
cars.sample(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
27945,24995,2016.0,ford f350,good,8.0,diesel,159083.0,automatic,truck,white,1.0,2018-12-30,13
39786,4250,NaN,ford escape,excellent,6.0,gas,134595.0,automatic,SUV,custom,1.0,2018-07-09,24
34705,6999,2012.0,chrysler 300,good,6.0,gas,NaN,automatic,sedan,black,NaN,2018-12-22,25
19073,7100,2008.0,gmc yukon,good,6.0,gas,210000.0,automatic,SUV,blue,1.0,2018-10-28,35
10742,5990,2009.0,toyota camry le,excellent,4.0,gas,NaN,automatic,sedan,white,NaN,2019-03-29,34
25600,8900,1999.0,jeep wrangler,excellent,6.0,gas,200000.0,automatic,SUV,NaN,1.0,2018-08-06,9
21978,19500,2011.0,toyota tacoma,good,6.0,gas,124000.0,automatic,pickup,black,1.0,2018-06-27,6
13416,5200,2009.0,toyota camry,good,4.0,gas,143000.0,automatic,sedan,grey,NaN,2018-07-21,56
16100,18600,2012.0,ford mustang gt coupe 2d,good,8.0,gas,96562.0,other,coupe,red,NaN,2018-07-04,28
21646,16995,2014.0,ford f-150,excellent,6.0,gas,159025.0,automatic,pickup,white,1.0,2018-11-16,33


In [4]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [5]:
st.header('Car Data')
st.dataframe(cars)

2024-05-07 19:13:29.924 
  command:

    streamlit run c:\Personal Files\Coding\Anaconda3\envs\standard\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [6]:
#cars['model_year'] = cars['model_year'].astype(int)
#cars['cylinders'] = cars['cylinders'].astype(int)
cars['is_4wd'] = cars['is_4wd'].astype(str).replace('1.0', 'Yes').fillna('No')

This is a pretty interesting dataset, and there are a number of possibilities for the data analysis that can be done. I would like to compare the prices cars are sold for based on a few different sets of data to get an idea as to how those markers affect the price of a car. In particular, I'd like to explore the relationships the car's price has with its mileage (odometer), the type of car sold, and the length that each car was listed. This could gleam some information as to the conditions under which cars are sold for the highest prices. 

Lots of the ploty.express graphs are very suitable for this with the right data. I will try to do some comparing and contrasting of the infromation conveyed between plots and charts from that library and the ones used for other TripleTen work. As such, some additional grouping will be needed.

In [7]:
mileage = cars.groupby('price')['odometer'].mean().reset_index() #These methods are used to drop the NA values
mileage.sample(5)

,price,odometer
1278,11485,52680.000000
156,1695,239380.333333
857,7974,74532.000000
2548,25863,62705.000000
1325,11880,203820.000000


In [8]:
mileage_price = mileage.groupby('price')
mileage_price
mileage_price.sample()

,price,odometer
0,1,40767.195556
1,3,NaN
2,5,NaN
3,6,173500.000000
4,9,119.000000
...,...,...
3438,145000,140000.000000
3439,175000,149000.000000
3440,189000,151248.000000
3441,300000,NaN


In [9]:
car_type = cars.groupby('type')['price'].mean().reset_index()
car_type.sample(5)

,type,price
6,offroad,14292.294393
11,van,10546.941548
9,sedan,6965.358647
3,coupe,14353.442901
0,SUV,11149.400000


In [10]:
car_type_price = car_type.groupby('price')
car_type_price.sample()

,type,price
4,hatchback,6868.513849
9,sedan,6965.358647
5,mini-van,8193.177433
12,wagon,9088.134328
11,van,10546.941548
7,other,10989.714844
0,SUV,11149.400000
6,offroad,14292.294393
3,coupe,14353.442901
2,convertible,14575.881166


In [11]:
list_length = cars.groupby('days_listed')['price'].mean().reset_index()
list_length.sample(5)

,days_listed,price
172,172,11937.400000
123,123,12162.000000
221,252,11500.000000
189,189,13998.000000
106,106,14145.460317


In [12]:
list_length_price = list_length.groupby('price')
list_length_price.sample()

,days_listed,price
212,223,1500.0
195,195,1600.5
209,213,1975.5
188,188,2995.0
223,261,3800.0
...,...,...
202,204,25988.0
197,199,27500.0
218,240,31995.0
194,194,34000.0


I initally planned to also compare the differences in plots between pyplot and plotly, but pyplot gave me some issues. So, we'll be focusing on the plotly.express charts, which should still give plenty of information.

In [13]:
st.header('price by mileage')
#cars_json = cars.to_json()
mp_hist = px.histogram(cars, x='price', y='odometer', nbins=100)
st.write(mp_hist)


In [20]:
st.header('price by type')
type_hist = px.histogram(cars, x='price', y='type', nbins=10)
st.write(type_hist)

In [21]:
st.header('price by list time')
time_hist = px.histogram(cars, x='price', y='days_listed', nbins=100)
st.write(time_hist)

In [22]:
st.header('price by year')
year_scatt = px.scatter(cars, x='price', y='days_listed', color='model_year')
st.write(year_scatt)

I've gotten some code that works after a healthy amount of fiddling. There's a lot of code I didn't end up using, and my plans for this project had to change on the fly a bit. Maybe it'll be good practice for the real thing. I will keep the code in for review as a means of showing my prior work.

This space will have observations on the data once I have loaded up a page so that I coul take a look at them. If a reveiwer is seeing this, then it's likely because I haven't seen the actual charts for some reason.